In [1]:
import os
cwd = os.getcwd()
data_dir = cwd + '/dataset'
os.listdir(data_dir)

['prev_check2_clean.csv',
 'application_test.csv',
 '.DS_Store',
 'HomeCredit_columns_description.csv',
 'POS_CASH_balance.csv',
 'train_set.csv',
 'test_set.csv',
 'credit_card_balance.csv',
 'installments_payments.csv',
 'application_train.csv',
 'bureau.csv',
 'bu_clean.csv',
 'prev_check1_clean.csv',
 'previous_application.csv',
 'HomeCredit_columns_description.xls',
 'bureau_balance.csv',
 'sample_submission.csv']

In [129]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import seaborn as sns

import lightgbm as lgb
from sklearn.cross_validation import train_test_split,KFold
from sklearn.metrics import auc
from sklearn.model_selection import GridSearchCV
import sklearn 
#5 classifiers for stacking
from sklearn.ensemble import (RandomForestRegressor,AdaBoostRegressor,\
                             GradientBoostingRegressor,ExtraTreesRegressor)
from sklearn.svm import SVC


%matplotlib inline
pd.set_option('max_columns',500)
pd.set_option('max_rows',500)

In [3]:
train_set = pd.read_csv(data_dir + "/train_set.csv",index_col='SK_ID_CURR')
test_set = pd.read_csv(data_dir + "/test_set.csv",index_col='SK_ID_CURR')

In [4]:
y_train = train_set["TARGET"].values
x_train = train_set.drop(["TARGET"],axis = 1).values
x_test = test_set.values

In [9]:
X_train, X_valid, Y_train, Y_valid = train_test_split(x_train, y_train, test_size=0.2, random_state=10)

In [5]:
#Class for extending sklearn classifier
class SklearnWrapper(object):
    def __init__(self, clf, seed=0, params = None):
        params["random_state"] = seed
        self.clf = clf(**params)
        
    def train(self, x_train, y_train):
        self.clf.fit(x_train,y_train)
        
    def predict(self, x):
        return self.clf.predict(x)
    
    def fit(self, x, y):
        return self.clf.fit(x,y)
    
    def feature_importances(self,x,y):
        return self.clf.fit(x,y).feature_importances_

In [112]:
# Random Forest parameters
rf_params = {
    'n_jobs': -1, 
    'n_estimators': 50,
    'warm_start': False, 
    'max_features': 0.15,
    'random_state':0,
    #'max_depth': 6,
    'max_depth': 10,
    'min_samples_leaf': 5,
    #'max_features' : 'sqrt',
    'verbose': 0
}
# Extra Trees Parameters
et_params = {
    'n_jobs': -1,
    'n_estimators':500,
    #'max_features': 0.5,
    'max_depth': 8,
    'min_samples_leaf': 2,
    'verbose': 0
}
# AdaBoost parameters
ada_params = {
    'n_estimators': 500,
    #'learning_rate' : 0.75
    'learning_rate' : 0.5
}
# Gradient Boosting parameters
gb_params = {
    'n_estimators': 500,
     #'max_features': 0.2,
    #'max_depth': 5,
    'max_depth': 8,
    'min_samples_leaf': 2,
    'verbose': 0
}
# Support Vector Classifier parameters 
svc_params = {
    'kernel' : 'linear',
    #'C' : 0.025
    'C':0.05
}

In [113]:
model = [(RandomForestRegressor,rf_params),
         (AdaBoostRegressor,ada_params),
         (GradientBoostingRegressor,gb_params),
         (ExtraTreesRegressor,et_params)]

In [133]:
rf_search_dict = {
    'n_estimators': [i for i in range(50,101,10)],
    'max_features': [i/100 for i in range(10,51,5)],
    'max_depth': [i for i in range(4,11,2)],
    'min_samples_leaf': [i for i in range(2,6)]
}
gsearch = GridSearchCV(estimator=RandomForestRegressor(), param_grid=rf_search_dict, 
                       scoring='roc_auc', cv=3)

rf_params['random_state'] = 0
rf_model = RandomForestRegressor(**rf_params)

In [ ]:
#rf_model.fit(X_train,Y_train)
gsearch.fit(X_train,Y_train)

In [116]:
h_valid = rf_model.predict(X_valid)
h_train = rf_model.predict(X_train)

In [117]:
sklearn.metrics.roc_auc_score(Y_valid,h_valid)

0.7334376968326555

In [118]:
sklearn.metrics.roc_auc_score(Y_train,h_train)

0.8030506533067198

In [126]:
[i/100 for i in range(10,50,5)]

[0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45]

range(0, 1)